In [ ]:
import ee
from geoEpic.gee import CompositeCollection

In [ ]:
# conda create --name geo_epic python=3.11.9
# conda activate geo_epic
# pip install git+https://github.com/smarsGroup/geo-epic.git

In [49]:
col =  CompositeCollection('./HLS.yml')

In [ ]:
#Extract hls the time series of a location
df = col.extract([[-98.114, 41.855]])
df

In [ ]:
hls_ee = col.merged()
# hls_ee can be used as a normal earth engine collection.
hls_ee


In [53]:
# Below is an eaxmple of calculating and exporting ndvi.
# Filter collection to a rectangle in Nebraska
nebraska = ee.Geometry.Rectangle([-99.5, 41.0, -99.0, 41.5])
hls_ee_filtered =  hls_ee.filterBounds(nebraska)
   
# Function to calculate NDVI
def calc_NDVI(image):
    ndvi = image.normalizedDifference(['nir', 'red']).rename('NDVI')
    return ndvi

# Map the NDVI function over the collection
NDVI = hls_ee_filtered.map(calc_NDVI)

In [54]:
# clip NDVI and export to google drive
clippedNDVI = NDVI.mean().clip(nebraska)

task = ee.batch.Export.image.toDrive(
    image=clippedNDVI,
    description='Nebraska_NDVI_2020',
    scale=30,
    region=nebraska)
task.start()

In [2]:
#check status to see if export is completed
task.status()